In [14]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.optimize import minimize 
from datetime import datetime as dt

In [15]:
@njit
def heston_charfunc(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):
    
    # constants
    a = kappa*theta
    b = kappa+lambd
    
    # common terms w.r.t phi
    rspi = rho*sigma*phi*1j
    
    # define d parameter given phi and b
    d = np.sqrt( (rho*sigma*phi*1j - b)**2 + (phi*1j+phi**2)*sigma**2 )
    
    # define g parameter given phi, b and d
    g = (b-rspi+d)/(b-rspi-d)
    
    # calculate characteristic function by components
    exp1 = np.exp(r*phi*1j*tau)
    term2 = S0**(phi*1j) * ( (1-g*np.exp(d*tau))/(1-g) )**(-2*a/sigma**2)
    exp2 = np.exp(a*tau*(b-rspi+d)/sigma**2 + v0*(b-rspi+d)*( (1-np.exp(d*tau))/(1-g*np.exp(d*tau)) )/sigma**2)
    return exp1*term2*exp2

@jit(forceobj=True)
def heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
    
    P, umax, N = 0, 100, 10000
    dphi=umax/N #dphi is width
    for i in range(1,N):
        # rectangular integration
        phi = dphi * (2*i + 1)/2 # midpoint to calculate height
        numerator = np.exp(r*tau)*heston_charfunc(phi-1j,*args) - K * heston_charfunc(phi,*args)
        denominator = 1j*phi*K**(1j*phi)
        
        P += dphi * numerator/denominator
        
    return np.real((S0 - K*np.exp(-r*tau))/2 + P/np.pi)

In [16]:
# Parameters to test model
S0 = 100. # initial asset price
K = 100. # strike
v0 = 0.1 # initial variance
r = 0.03 # risk free rate
kappa = 1.5768 # rate of mean reversion of variance process
theta = 0.0398 # long-term mean variance
sigma = 0.3 # volatility of volatility
lambd = 0.575 # risk premium of variance
rho = -0.5711 # correlation between variance and stock process
tau = 1. # time to maturity
heston_price_rec( S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r )

11.521145092601866

In [17]:
file = '../data/processed_data/2020_2022_moneyness_filtere.csv'
df_read = pd.read_csv(file)

cal_date = '2021-01-04'
test_date = '2021-01-05'
df_cal = df_read[df_read['Quote_date'] == cal_date]
df_test = df_read[df_read['Quote_date'] == test_date]

print(f'Size of calibration set: {df_cal.shape[0]}')
print(f'Size of test set: {df_test.shape[0]}')
print(df_cal.head(1))


Size of calibration set: 6287
Size of test set: 6508
         Unnamed: 0  Unnamed: 0.1  Quote_date     Price  Underlying_last  \
1532925     8400702       8400702  2021-01-04  2002.205          3701.38   

         Strike  TTM     R  Moneyness  
1532925  1700.0    2  0.09   2.177282  


In [18]:

# Parameters to test model
S0 = df_cal['Underlying_last'].to_numpy('float')
K = df_cal['Strike'].to_numpy('float')
r = df_cal['R'].to_numpy('float')
tau = df_cal['TTM'].to_numpy('float')
P = df_cal['Price'].to_numpy('float')

params = {"v0": {"x0": 0.1, "lbub": [1e-3,0.1]}, 
          "kappa": {"x0": 3, "lbub": [1e-3,5]},
          "theta": {"x0": 0.05, "lbub": [1e-3,0.1]},
          "sigma": {"x0": 0.3, "lbub": [1e-2,1]},
          "rho": {"x0": -0.8, "lbub": [-1,0]},
          "lambd": {"x0": 0.03, "lbub": [-1,1]},
          }
x0 = [param["x0"] for key, param in params.items()]
bnds = [param["lbub"] for key, param in params.items()]
def SqErr(x):
    v0, kappa, theta, sigma, rho, lambd = [param for param in x]
    err = np.sum( (P-heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r))**2 /len(P) )
    
    # Zero penalty term - no good guesses for parameters
    pen = 0 #np.sum( [(x_i-x0_i)**2 for x_i, x0_i in zip(x, x0)] )
    print('Err:', err)
          
    return err + pen
result = minimize(SqErr, x0, tol = 1e-3, method='SLSQP', options={'maxiter': 1e4 }, bounds=bnds)
v0, kappa, theta, sigma, rho, lambd = [param for param in result.x]
v0, kappa, theta, sigma, rho, lambd

KeyboardInterrupt: 

In [ ]:
# Parameters to test model
S0 = df_cal['Underlying_last'].to_numpy('float')
K = df_cal['Strike'].to_numpy('float')
r = df_cal['R'].to_numpy('float')
tau = df_cal['TTM'].to_numpy('float')
P = df_cal['Price'].to_numpy('float')

params = {"v0": {"x0": 0.1, "lbub": [1e-3,0.1]}, 
          "kappa": {"x0": 3, "lbub": [1e-3,5]},
          "theta": {"x0": 0.05, "lbub": [1e-3,0.1]},
          "sigma": {"x0": 0.3, "lbub": [1e-2,1]},
          "rho": {"x0": -0.8, "lbub": [-1,0]},
          "lambd": {"x0": 0.03, "lbub": [-1,1]},
          }
x0 = [param["x0"] for key, param in params.items()]
bnds = [param["lbub"] for key, param in params.items()]
def SqErr(x):
    v0, kappa, theta, sigma, rho, lambd = [param for param in x]
    
    # Attempted to use scipy integrate quad module as constrained to single floats not arrays
    err = np.sum([ (P_i-heston_price(S0, K_i, v0, kappa, theta, sigma, rho, lambd, tau_i, r_i))**2 /len(P) \
                   for P_i, K_i, tau_i, r_i in zip(marketPrices, K, tau, r)])
    
    # Decided to use rectangular integration function in the end
    # err = np.sum( (P-heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r))**2 /len(P) )
    
    # Zero penalty term - no good guesses for parameters
    pen = 0 #np.sum( [(x_i-x0_i)**2 for x_i, x0_i in zip(x, x0)] )
          
    return err + pen

result = minimize(SqErr, x0, tol = 1e-3, method='SLSQP', options={'maxiter': 1e4 }, bounds=bnds)
v0, kappa, theta, sigma, rho, lambd = [param for param in result.x]
v0, kappa, theta, sigma, rho, lambd

NameError: name 'df_cal' is not defined

In [19]:
import numpy as np
# Parallel computation using numba
from numba import jit, njit, prange 
from numba import cuda
i = complex(0,1)

# Optimizer
from lmfit import Parameters, minimize
from scipy.optimize import dual_annealing

from time import time


In [20]:

# To be used in the Heston pricer
@jit
def fHeston(s, St, K, r, T, sigma, kappa, theta, volvol, rho):
    # To be used a lot
    prod = rho * sigma *i *s 
    
    # Calculate d
    d1 = (prod - kappa)**2
    d2 = (sigma**2) * (i*s + s**2)
    d = np.sqrt(d1 + d2)
    
    # Calculate g
    g1 = kappa - prod - d
    g2 = kappa - prod + d
    g = g1/g2
    
    # Calculate first exponential
    exp1 = np.exp(np.log(St) * i *s) * np.exp(i * s* r* T)
    exp2 = 1 - g * np.exp(-d *T)
    exp3 = 1- g
    mainExp1 = exp1*np.power(exp2/exp3, -2*theta*kappa/(sigma **2))
    
    # Calculate second exponential
    exp4 = theta * kappa * T/(sigma **2)
    exp5 = volvol/(sigma **2)
    exp6 = (1 - np.exp(-d * T))/(1 - g * np.exp(-d * T))
    mainExp2 = np.exp((exp4 * g1) + (exp5 *g1 * exp6))
    
    return (mainExp1 * mainExp2)
# Heston Pricer (allow for parallel processing with numba)
@jit()
def priceHestonMid(St, K, r, T, sigma, kappa, theta, volvol, rho):
    start = time()
    P, iterations, maxNumber = 0,1000,100
    ds = maxNumber/iterations
    
    element1 = 0.5 * (St - K * np.exp(-r * T))
    
    # Calculate the complex integral
    # Using j instead of i to avoid confusion
    for j in prange(1, iterations):
        s1 = ds * (2*j + 1)/2
        s2 = s1 - i
        
        numerator1 = fHeston(s2,  St, K, r, T, 
                             sigma, kappa, theta, volvol, rho)
        numerator2 = K * fHeston(s1,  St, K, r, T, 
                              sigma, kappa, theta, volvol, rho)
        denominator = np.exp(np.log(K) * i * s1) *i *s1
        
        P = P + ds *(numerator1 - numerator2)/denominator
    
    element2 = P/np.pi
    print(f'Mid return= {np.real((element1 + element2))}')
    x = np.real((element1 + element2))
    print('Mid rect time = ', time()-start)
    return x

@njit
def integrand(s, St, K, r, T, sigma, kappa, theta, volvol, rho):
    numerator1 = fHeston(s,  St, K, r, T, 
                             sigma, kappa, theta, volvol, rho)
    numerator2 = K * fHeston(s,  St, K, r, T, 
                              sigma, kappa, theta, volvol, rho)
    denominator = np.exp(np.log(K) * i * s) *i *s
    x = np.real((numerator1 - numerator2)/denominator)
    return x

# Heston Pricer (allow for parallel processing with numba)
@jit(forceobj=True)
def priceHestonIntegral(St, K, r, T, sigma, kappa, theta, volvol, rho):
    start = time()
    integral = np.real([quad(integrand, 0, np.inf, args=(St, K_i, r_i, T_i, sigma, kappa, theta, volvol, rho))[0] for K_i, r_i, T_i in zip(K, r, T) ])
    x = 0.5 * (St - K * np.exp(-r * T)) + integral/np.pi
    print('Integral time = ', time()-start)
    return x
    

In [21]:
# Can be used for debugging
def iter_cb(params, iter, resid):
    parameters = [params['sigma'].value, 
                  params['kappa'].value, 
                  params['theta'].value, 
                  params['volvol'].value, 
                  params['rho'].value, 
                  np.sum(np.square(resid))]
    print(parameters) 


#Initialize parameters
sigma, kappa, theta, volvol, rho = 0.1, 0.1, 0.1, 0.1, 0.1
marketPrices = df_cal['Price'].to_numpy('float')
strikes = df_cal['Strike'].to_numpy('float')
rates = df_cal['R'].to_numpy('float')
maturities = df_cal['TTM'].to_numpy('float')

In [52]:
# This is the calibration function
#def calibratorHeston(St, initialValues = [0.3,3,0.05,0.03,-0.8], 
#                              lowerBounds = [1e-2,1e-3,1e-3,-1,-1], 
#                              upperBounds = [1,5,0.1,1,0]):
#def calibratorHeston(St, initialValues = [0.5,0.5,0.5,0.5,-0.5], 
#                              lowerBounds = [1e-2,1e-2,1e-2,1e-2,-1], 
#                              upperBounds = [10,10,10,10,0]):
def calibratorHeston(St, initialValues = [8.509993434100602,3.0448842720730442,3.530394881283103,1.4421606996525653,-0.14647153232831012], 
                              lowerBounds = [1e-2,1e-2,1e-2,1e-2,-1], 
                              upperBounds = [10,10,10,10,0]):

    '''Implementation of the Levenberg Marquardt algorithm in Python to find the optimal value 
        based on a given volatility surface.
        
        Function to be minimized:
            Error = (MarketPrices - ModelPrices)/MarketPrices
        
        INPUTS
        ===========
        1) Volatility Surface
            - Obtained from webscrapping. 
        
        2) Risk Free Curve
            - Obtained from webscrapping. 
            
        3) initialValues
            - Initialization values for the algorithms in this order:
                [sigma, kappa, theta, volvol, rho]
                
            - Default value: [0.1,0.1,0.1,0.1,0.1]
            
        4) lowerBounds
            -Fix lower limit for the values
            - Default value: [0.001,0.001,0.001,0.001,-1.00]
            
        5) upperBounds
            -Fix upper limit for the values
            - Default value: [1.0,1.0,1.0,1.0,1.0]
            
        6) St is the stock price today.
    
    Set Up
    =======
    1) We define the limits of the parameters using the Parameters object
    2) We define an objective function that gives the relative difference between market prices and model prices
    3) We minimize the function using the Levenberg Marquardt algorithm
    '''
        
    '''1) Define parameters
    =========================='''
    params = Parameters()
    params.add('sigma',value = initialValues[0], min = lowerBounds[0], max = upperBounds[0])
    params.add('kappa',value = initialValues[1], min = lowerBounds[1], max = upperBounds[1])
    params.add('theta',value = initialValues[2], min = lowerBounds[2], max = upperBounds[2])
    params.add('volvol', value = initialValues[3], min = lowerBounds[3], max = upperBounds[3])
    params.add('rho', value = initialValues[4], min = lowerBounds[4], max = upperBounds[4])
    print(initialValues[0], initialValues[1], initialValues[2], initialValues[3], initialValues[4])
    
    '''2) Define objective function
    ================================'''
    objectiveFunctionHeston = lambda paramVect: (marketPrices - priceHestonMid(St, strikes,  
                                                                        rates, 
                                                                        maturities, 
                                                                        paramVect['sigma'].value,                         
                                                                        paramVect['kappa'].value,
                                                                        paramVect['theta'].value,
                                                                        paramVect['volvol'].value,
                                                                        paramVect['rho'].value))/marketPrices   
    
    '''3) Optimize parameters
    =============================='''
    result = minimize(objectiveFunctionHeston, 
                      params, 
                      method = 'leastsq',
                      iter_cb = iter_cb,
                      ftol = 1e-6) 
    return(result)

calibratorHeston(3701.38)

8.509993434100602 3.0448842720730442 3.530394881283103 1.4421606996525653 -0.14647153232831012


C:\Users\Erlend\AppData\Local\Temp\ipykernel_28936\3696390152.py:31: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "priceHestonMid" failed type inference due to: Untyped global name 'time': Cannot determine Numba type of <class 'builtin_function_or_method'>

File "..\..\..\..\..\..\..\AppData\Local\Temp\ipykernel_28936\3696390152.py", line 33:
<source missing, REPL/exec in use?>

  @jit()
C:\Users\Erlend\AppData\Local\Temp\ipykernel_28936\3696390152.py:31: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "priceHestonMid" failed type inference due to: Untyped global name 'time': Cannot determine Numba type of <class 'builtin_function_or_method'>

File "..\..\..\..\..\..\..\AppData\Local\Temp\ipykernel_28936\3696390152.py", line 33:
<source missing, REPL/exec in use?>

  @jit()
c:\Users\Erlend\AppData\Local\Programs\Python\Python39\lib\site-packages\numba\core\object_mode_passes.py

Mid return= [2.85001089e+03 2.82498472e+03 2.80026120e+03 ... 1.58407201e+56
 1.63553039e+56 1.68586485e+56]
Mid rect time =  15.690959215164185
[8.509993434100602, 3.0448842720730442, 3.530394881283103, 1.4421606996525653, -0.14647153232831012, 1.0919845207485008e+109]
Mid return= [2.85001089e+03 2.82498472e+03 2.80026120e+03 ... 1.58407201e+56
 1.63553039e+56 1.68586485e+56]
Mid rect time =  11.499532699584961
[8.509993434100602, 3.0448842720730442, 3.530394881283103, 1.4421606996525653, -0.14647153232831012, 1.0919845207485008e+109]
Mid return= [2.85001089e+03 2.82498472e+03 2.80026120e+03 ... 1.58407201e+56
 1.63553039e+56 1.68586485e+56]
Mid rect time =  11.10593557357788
[8.509993434100602, 3.0448842720730442, 3.530394881283103, 1.4421606996525653, -0.14647153232831012, 1.0919845207485008e+109]
Mid return= [2.85001089e+03 2.82498471e+03 2.80026120e+03 ... 1.58407773e+56
 1.63553610e+56 1.68587056e+56]
Mid rect time =  10.447930812835693
[8.509993475346523, 3.0448842720730442, 3.5

KeyboardInterrupt: 

In [22]:
def calibratorHeston(St, initialValues = [8.509993434100602,3.0448842720730442,3.530394881283103,1.4421606996525653,-0.14647153232831012], 
                              lowerBounds = [1e-2,1e-2,1e-2,1e-2,-1], 
                              upperBounds = [10,10,10,10,0]):

    '''Implementation of the Levenberg Marquardt algorithm in Python to find the optimal value 
        based on a given volatility surface.
        
        Function to be minimized:
            Error = (MarketPrices - ModelPrices)/MarketPrices
        
        INPUTS
        ===========
        1) Volatility Surface
            - Obtained from webscrapping. 
        
        2) Risk Free Curve
            - Obtained from webscrapping. 
            
        3) initialValues
            - Initialization values for the algorithms in this order:
                [sigma, kappa, theta, volvol, rho]
                
            - Default value: [0.1,0.1,0.1,0.1,0.1]
            
        4) lowerBounds
            -Fix lower limit for the values
            - Default value: [0.001,0.001,0.001,0.001,-1.00]
            
        5) upperBounds
            -Fix upper limit for the values
            - Default value: [1.0,1.0,1.0,1.0,1.0]
            
        6) St is the stock price today.
    
    Set Up
    =======
    1) We define the limits of the parameters using the Parameters object
    2) We define an objective function that gives the relative difference between market prices and model prices
    3) We minimize the function using the Levenberg Marquardt algorithm
    '''
        
    '''1) Define parameters
    =========================='''
    params = Parameters()
    params.add('sigma',value = initialValues[0], min = lowerBounds[0], max = upperBounds[0])
    params.add('kappa',value = initialValues[1], min = lowerBounds[1], max = upperBounds[1])
    params.add('theta',value = initialValues[2], min = lowerBounds[2], max = upperBounds[2])
    params.add('volvol', value = initialValues[3], min = lowerBounds[3], max = upperBounds[3])
    params.add('rho', value = initialValues[4], min = lowerBounds[4], max = upperBounds[4])
    print(initialValues[0], initialValues[1], initialValues[2], initialValues[3], initialValues[4])
    
    '''2) Define objective function
    ================================'''
    objectiveFunctionHeston = lambda paramVect: (marketPrices - priceHestonIntegral(St, strikes,  
                                                                        rates, 
                                                                        maturities, 
                                                                        paramVect['sigma'].value,                         
                                                                        paramVect['kappa'].value,
                                                                        paramVect['theta'].value,
                                                                        paramVect['volvol'].value,
                                                                        paramVect['rho'].value))/marketPrices   
    
    '''3) Optimize parameters
    =============================='''
    result = minimize(objectiveFunctionHeston, 
                      params, 
                      method = 'leastsq',
                      iter_cb = iter_cb,
                      ftol = 1e-6) 
    return(result)

calibratorHeston(3701.38)

8.509993434100602 3.0448842720730442 3.530394881283103 1.4421606996525653 -0.14647153232831012
Integral time =  2.746678352355957
[8.509993434100602, 3.0448842720730442, 3.530394881283103, 1.4421606996525653, -0.14647153232831012, 562632075174.2982]
Integral time =  1.8950936794281006
[8.509993434100602, 3.0448842720730442, 3.530394881283103, 1.4421606996525653, -0.14647153232831012, 562632075174.2982]
Integral time =  2.0299036502838135
[8.509993434100602, 3.0448842720730442, 3.530394881283103, 1.4421606996525653, -0.14647153232831012, 562632075174.2982]
Integral time =  2.3943488597869873
[8.509993475346523, 3.0448842720730442, 3.530394881283103, 1.4421606996525653, -0.14647153232831012, 562632074761.158]
Integral time =  2.1883726119995117
[8.509993434100602, 3.0448842996803847, 3.530394881283103, 1.4421606996525653, -0.14647153232831012, 562632075959.3074]
Integral time =  2.0964229106903076
[8.509993434100602, 3.0448842720730442, 3.530394902594657, 1.4421606996525653, -0.146471532

C:\Users\Erlend\AppData\Local\Temp\ipykernel_34980\3613260230.py:54: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  objectiveFunctionHeston = lambda paramVect: (marketPrices - priceHestonIntegral(St, strikes,


Integral time =  8.739360809326172
[9.172673938179434, 0.5336228569607775, 0.11227039385423651, 9.999311089395468, -0.5708923828397314, 70088375608.9034]
Integral time =  7.0255208015441895
[9.172670933212279, 0.5336228569607775, 0.11227039385423651, 9.999311089395468, -0.5708923828397314, 70088429552.75182]
Integral time =  6.1530516147613525
[9.172673938179434, 0.533621673702523, 0.11227039385423651, 9.999311089395468, -0.5708923828397314, 70088380229.87599]
Integral time =  7.972795248031616
[9.172673938179434, 0.5336228569607775, 0.11227097925906102, 9.999311089395468, -0.5708923828397314, 70088418101.45258]
Integral time =  7.104907512664795
[9.172673938179434, 0.5336228569607775, 0.11227039385423651, 9.999311060289717, -0.5708923828397314, 70088375123.19946]
Integral time =  8.643248081207275
[9.172673938179434, 0.5336228569607775, 0.11227039385423651, 9.999311089395468, -0.5708922912080787, 70088388920.62845]
Integral time =  5.595400810241699
[5.582428587288078, 2.9936453351426

In [ ]:
!pip install -U ipykernel

     -------------------------------------- 150.0/150.0 kB 4.5 MB/s eta 0:00:00
     -------------------------------------- 117.4/117.4 kB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 93.2/93.2 kB ? eta 0:00:00
     ---------------------------------------- 4.8/4.8 MB 11.5 MB/s eta 0:00:00


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.0.5
    Uninstalling traitlets-5.0.5:
      Successfully uninstalled traitlets-5.0.5
  Attempting uninstall: jupyter-core
    Found existing installation: jupyter-core 4.7.1
    Uninstalling jupyter-core-4.7.1:
      Successfully uninstalled jupyter-core-4.7.1
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.5.5
    Uninstalling ipykernel-5.5.5:
      Successfully uninstalled ipykernel-5.5.5
